In [4]:
import json
import glob

from IPython.display import Image, display
import pandas as pd
import multiprocess
from tqdm.contrib.concurrent import process_map

pool = multiprocess.Pool()


TRAIN_PATH = "../data/train/annotations/*.json"
ANNOTATIONS_FILE = "annotations.pkl"

In [2]:
def load_json_file(path):
    with open(path) as f:
        data = json.load(f)
        data['image'] = path.split("/")[-1][:-5] + ".jpg"
        return data

json_files = glob.glob(TRAIN_PATH)
train_df = pd.DataFrame.from_records(pool.map(load_json_file, json_files, chunksize=100))
train_df.head()

,source,chart-type,plot-bb,text,axes,data-series,image
0,generated,scatter,"{'height': 231, 'width': 416, 'x0': 57, 'y0': 65}","[{'id': 0, 'polygon': {'x0': 85, 'x1': 478, 'x...","{'x-axis': {'ticks': [{'id': 9, 'tick_pt': {'x...","[{'x': 1949.4200576738224, 'y': 66.68303958393...",e91e28111e86.jpg
1,generated,vertical_bar,"{'height': 164, 'width': 393, 'x0': 68, 'y0': 37}","[{'id': 0, 'polygon': {'x0': 109, 'x1': 355, '...","{'x-axis': {'ticks': [{'id': 11, 'tick_pt': {'...","[{'x': 'Jordan', 'y': 88.35159235668789}, {'x'...",75c0449f6917.jpg
2,generated,scatter,"{'height': 203, 'width': 409, 'x0': 54, 'y0': 21}","[{'id': 0, 'polygon': {'x0': 113, 'x1': 383, '...","{'x-axis': {'ticks': [{'id': 3, 'tick_pt': {'x...","[{'x': 0.9445463278316278, 'y': 52.13870797394...",66dd2a250237.jpg
3,generated,vertical_bar,"{'height': 165, 'width': 396, 'x0': 89, 'y0': 60}","[{'id': 0, 'polygon': {'x0': 71, 'x1': 465, 'x...","{'x-axis': {'ticks': [{'id': 14, 'tick_pt': {'...","[{'x': 'Cuba', 'y': 75695.38632268383}, {'x': ...",58595c30beab.jpg
4,generated,scatter,"{'height': 165, 'width': 394, 'x0': 79, 'y0': 69}","[{'id': 0, 'polygon': {'x0': 75, 'x1': 461, 'x...","{'x-axis': {'ticks': [{'id': 12, 'tick_pt': {'...","[{'x': 0.8933632249058832, 'y': 5.440194292653...",497a547454d7.jpg


In [5]:
train_df.to_pickle(ANNOTATIONS_FILE)

In [ ]:
display(train_df.source.value_counts().reset_index())
# sns.countplot(y="source", data=train_df)
# plt.title("Chart source")
# plt.show()

display(train_df['chart-type'].value_counts().reset_index())
# sns.countplot(y="chart-type", data=train_df)
# plt.title("Chart source")
# plt.show()

display(train_df.groupby(['source','chart-type']).size().reset_index().pivot(index='source', columns='chart-type', values=0).fillna(0))

,index,source
0,generated,59460
1,extracted,1118


,index,chart-type
0,line,24942
1,vertical_bar,19189
2,scatter,11243
3,dot,5131
4,horizontal_bar,73


chart-type,dot,horizontal_bar,line,scatter,vertical_bar
source,,,,,
extracted,0.0,73.0,423.0,165.0,457.0
generated,5131.0,0.0,24519.0,11078.0,18732.0
